In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder

import pickle

/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Import dataset

In [2]:
data = pd.read_csv("../DataFormating/final.csv")

In [3]:
data.head()

,Away Team Goals,Away Team Name,Home Team Goals,Home Team Name,Year,home_rank,home_total_points,home_previous_points,home_rank_change,home_cur_year_avg,...,away_cur_year_avg,away_cur_year_avg_weighted,away_last_year_avg,away_last_year_avg_weighted,away_two_year_ago_avg,away_two_year_ago_weighted,away_three_year_ago_avg,away_three_year_ago_weighted,Home Avg Goals,Away Avg Goals
0,1.0,Sweden,2.0,Brazil,1990.0,8,0.0,55,-5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,Poland,1.0,Belgium,1990.0,18,0.0,47,-1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,England,1.0,Tunisia,1990.0,31,0.0,35,7,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,Mexico,3.0,Belgium,1990.0,18,0.0,47,-1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,Denmark,1.0,Germany,1990.0,1,0.0,57,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Set `X` and `y`

In [4]:
data.columns

Index(['Away Team Goals', 'Away Team Name', 'Home Team Goals',
       'Home Team Name', 'Year', 'home_rank', 'home_total_points',
       'home_previous_points', 'home_rank_change', 'home_cur_year_avg',
       'home_cur_year_avg_weighted', 'home_last_year_avg',
       'home_last_year_avg_weighted', 'home_two_year_ago_avg',
       'home_two_year_ago_weighted', 'home_three_year_ago_avg',
       'home_three_year_ago_weighted', 'away_rank', 'away_total_points',
       'away_previous_points', 'away_rank_change', 'away_cur_year_avg',
       'away_cur_year_avg_weighted', 'away_last_year_avg',
       'away_last_year_avg_weighted', 'away_two_year_ago_avg',
       'away_two_year_ago_weighted', 'away_three_year_ago_avg',
       'away_three_year_ago_weighted', 'Home Avg Goals', 'Away Avg Goals'],
      dtype='object')

In [5]:
X = data.drop(["Away Team Goals", "Home Team Goals"], axis=1)

y = []
for i in range(len(data)):
    home_team_goals = data.iloc[i]["Home Team Goals"]
    away_team_goals = data.iloc[i]["Away Team Goals"]
    
    if home_team_goals > away_team_goals:
        y.append(1)
    elif home_team_goals < away_team_goals:
        y.append(2)
    else:
        y.append(0)

In [6]:
# Test
assert len(X) == len(y)

### Encode textual features from the `X` dataset

In [7]:
word_cup_teams = [
    "Egypt",
    "Morocco",
    "Nigeria",
    "Senegal",
    "Tunisia",
    "Australia",
    "IR Iran",
    "Japan",
    "Korea DPR",
    "Saudi Arabia",
    "Belgium",
    "Croatia",
    "Denmark",
    "England",
    "France",
    "Germany",
    "Iceland",
    "Poland",
    "Portugal",
    "Russia",
    "Serbia",
    "Spain",
    "Sweden",
    "Switzerland",
    "Costa Rica",
    "Mexico",
    "Panama",
    "Argentina",
    "Brazil",
    "Colombia",
    "Peru",
    "Uruguay"
]

team_names = list(data["Home Team Name"].unique()) + list(data["Away Team Name"].unique()) + word_cup_teams

In [8]:
team_name_encoder = LabelEncoder().fit(team_names)

In [9]:
X["Home Team Name"] = team_name_encoder.transform(X["Home Team Name"])
X["Away Team Name"] = team_name_encoder.transform(X["Away Team Name"])

### Feature Selection

In [10]:
feature_names = []

COLUMNS = []
for column_name in X.columns:
    for feature_name in feature_names:
        if feature_name in column_name:
            COLUMNS.append(column_name)
            break

COLUMNS = [
    'Away Team Name',
    'Home Team Name',

    'home_rank',
    'home_total_points',
    'home_cur_year_avg',
    'home_cur_year_avg_weighted',
    
    'away_rank',
    'away_total_points',
    'away_cur_year_avg',
    'away_cur_year_avg_weighted',
    
    'Home Avg Goals',
    'Away Avg Goals'
]
X = X[COLUMNS]

In [11]:
COLUMNS

['Away Team Name',
 'Home Team Name',
 'home_rank',
 'home_total_points',
 'home_cur_year_avg',
 'home_cur_year_avg_weighted',
 'away_rank',
 'away_total_points',
 'away_cur_year_avg',
 'away_cur_year_avg_weighted',
 'Home Avg Goals',
 'Away Avg Goals']

# Traning Session

In [12]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2072 entries, 0 to 2071
Data columns (total 12 columns):
Away Team Name                2072 non-null int64
Home Team Name                2072 non-null int64
home_rank                     2072 non-null int64
home_total_points             2072 non-null float64
home_cur_year_avg             2072 non-null float64
home_cur_year_avg_weighted    2072 non-null float64
away_rank                     2072 non-null int64
away_total_points             2072 non-null float64
away_cur_year_avg             2072 non-null float64
away_cur_year_avg_weighted    2072 non-null float64
Home Avg Goals                2072 non-null float64
Away Avg Goals                2072 non-null float64
dtypes: float64(8), int64(4)
memory usage: 194.3 KB


In [13]:
X.describe()

,Away Team Name,Home Team Name,home_rank,home_total_points,home_cur_year_avg,home_cur_year_avg_weighted,away_rank,away_total_points,away_cur_year_avg,away_cur_year_avg_weighted,Home Avg Goals,Away Avg Goals
count,2072.000000,2072.000000,2072.000000,2072.000000,2072.000000,2072.000000,2072.000000,2072.000000,2072.000000,2072.000000,2072.000000,2072.000000
mean,15.340734,14.483108,24.229730,208.071535,106.249561,106.249561,26.265927,206.032640,105.409020,105.409020,0.910851,0.668214
std,10.089973,9.703808,20.723428,415.930367,217.116068,217.116068,22.644935,414.141492,215.688463,215.688463,1.060718,0.840626
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,6.000000,8.000000,0.000000,0.000000,0.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,16.000000,14.000000,19.000000,0.000000,0.000000,0.000000,20.000000,0.000000,0.000000,0.000000,0.750000,0.500000
75%,24.000000,23.000000,36.000000,0.000000,0.000000,0.000000,39.000000,0.000000,0.000000,0.000000,1.500000,1.000000
max,32.000000,32.000000,139.000000,1725.290000,1090.540000,1090.540000,139.000000,1725.290000,1090.540000,1090.540000,7.000000,6.000000


In [14]:
model = MLPClassifier(
            hidden_layer_sizes=(60,),
            activation="logistic",
            solver="adam",
            alpha=0.001
        )

In [15]:
model.fit(X, y)

MLPClassifier(activation='logistic', alpha=0.001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(60,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

# Save model and encoders

In [16]:
with open("model.b", "wb") as f:
    pickle.dump(model, f)
    
with open("team_name_encoder.b", "wb") as f:
    pickle.dump(team_name_encoder, f)